In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import bisect

import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm

# pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000
# pd.options.display.max_seq_items = 2000

In [9]:
training_df = pd.read_csv('df_log/228/training_self/training_self_0309.csv')
dev_df = pd.read_csv('df_log/228/training_self/dev_self_0314.csv')
test_df = pd.read_csv('df_log/228/training_self/test_self_0314.csv')
training_df = training_df.fillna(0)
dev_df = dev_df.fillna(0)
test_df = test_df.fillna(0)

In [3]:
five_folds = [[[0, 1, 2, 3], [4]], [[0, 1, 2, 4], [3]], [[0, 1, 3, 4], [2]], [[0, 2, 3, 4], [1]], [[1, 2, 3, 4], [0]]]

In [4]:
training_five_folds = []
for idx, fold in enumerate(five_folds):
    temp_df = pd.read_csv('df_log/228/training_self/training_self_0314_fold'+str(idx)+'.csv')
    training_five_folds.append(temp_df.fillna(0))

In [5]:
import lightgbm as lgb

In [7]:
drop_features = ['fold', 'label', 'FileID', 'first_time_occur', 'last_time_occur', 'customer_has_static_ratio', 'customer_has_static_b3_ratio', 'customer_has_static_count', 'customer_has_static_b3_count']
drop_features = ['fold', 'label', 'FileID', 'first_time_occur', 'last_time_occur']
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 15,
    'learning_rate': 0.05,
    'feature_fraction': 0.5,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'verbose': 0,
    'lambda_l2': 15.0,
    'min_gain_to_split': 0,
    'min_data_in_leaf': 15,
    'max_depth': 15
}

predict_results = []
clssfis = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    fold_training = pd.concat([training_five_folds[merge_folds[0]], training_five_folds[merge_folds[1]], \
                              training_five_folds[merge_folds[2]], training_five_folds[merge_folds[3]]])
    lgb_train = lgb.Dataset(fold_training.drop(drop_features, axis=1).values, fold_training['label'].values)
    lgb_eval = lgb.Dataset(dev_df.drop(drop_features, axis=1).values, dev_df['label'].values, reference=lgb_train)
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=80,
                valid_sets=lgb_eval)
    
    fpr, tpr, thresholds = roc_curve(fold_training['label'], gbm.predict(fold_training.drop(drop_features, axis=1).values, num_iteration=gbm.best_iteration))
    print(auc(fpr, tpr))
    fpr, tpr, thresholds = roc_curve(dev_df['label'], gbm.predict(dev_df.drop(drop_features, axis=1).values, num_iteration=gbm.best_iteration))
    print(auc(fpr, tpr))
    
    clssfis.append(gbm)
    predict_results.append(gbm.predict(dev_df.drop(drop_features, axis=1).values, num_iteration=gbm.best_iteration))

[1]	valid_0's l2: 0.230129	valid_0's auc: 0.855233
[2]	valid_0's l2: 0.212315	valid_0's auc: 0.873734
[3]	valid_0's l2: 0.196058	valid_0's auc: 0.890611
[4]	valid_0's l2: 0.181179	valid_0's auc: 0.910769
[5]	valid_0's l2: 0.167925	valid_0's auc: 0.911062
[6]	valid_0's l2: 0.156026	valid_0's auc: 0.911106
[7]	valid_0's l2: 0.145173	valid_0's auc: 0.9168
[8]	valid_0's l2: 0.135705	valid_0's auc: 0.92861
[9]	valid_0's l2: 0.126722	valid_0's auc: 0.929244
[10]	valid_0's l2: 0.11885	valid_0's auc: 0.928914
[11]	valid_0's l2: 0.111692	valid_0's auc: 0.933291
[12]	valid_0's l2: 0.105105	valid_0's auc: 0.933466
[13]	valid_0's l2: 0.0991698	valid_0's auc: 0.934178
[14]	valid_0's l2: 0.0938842	valid_0's auc: 0.934322
[15]	valid_0's l2: 0.0889579	valid_0's auc: 0.934449
[16]	valid_0's l2: 0.0844973	valid_0's auc: 0.934395
[17]	valid_0's l2: 0.080494	valid_0's auc: 0.934526
[18]	valid_0's l2: 0.0769141	valid_0's auc: 0.934119
[19]	valid_0's l2: 0.0736507	valid_0's auc: 0.940588
[20]	valid_0's l2: 

[79]	valid_0's l2: 0.0396703	valid_0's auc: 0.957221
[80]	valid_0's l2: 0.0396118	valid_0's auc: 0.957298
0.9729811733647677
0.9572978257129513
[1]	valid_0's l2: 0.230418	valid_0's auc: 0.849382
[2]	valid_0's l2: 0.212617	valid_0's auc: 0.872272
[3]	valid_0's l2: 0.196379	valid_0's auc: 0.913027
[4]	valid_0's l2: 0.181568	valid_0's auc: 0.924273
[5]	valid_0's l2: 0.168219	valid_0's auc: 0.9276
[6]	valid_0's l2: 0.156272	valid_0's auc: 0.928808
[7]	valid_0's l2: 0.145437	valid_0's auc: 0.929138
[8]	valid_0's l2: 0.135916	valid_0's auc: 0.935895
[9]	valid_0's l2: 0.126938	valid_0's auc: 0.936336
[10]	valid_0's l2: 0.119047	valid_0's auc: 0.935891
[11]	valid_0's l2: 0.112018	valid_0's auc: 0.937133
[12]	valid_0's l2: 0.105439	valid_0's auc: 0.93699
[13]	valid_0's l2: 0.0994911	valid_0's auc: 0.937057
[14]	valid_0's l2: 0.0942012	valid_0's auc: 0.936204
[15]	valid_0's l2: 0.0892504	valid_0's auc: 0.936196
[16]	valid_0's l2: 0.0847484	valid_0's auc: 0.936356
[17]	valid_0's l2: 0.0806974	val

0.973535990300643
0.9572973979240402
[1]	valid_0's l2: 0.230316	valid_0's auc: 0.884483
[2]	valid_0's l2: 0.212581	valid_0's auc: 0.894587
[3]	valid_0's l2: 0.196426	valid_0's auc: 0.905868
[4]	valid_0's l2: 0.181593	valid_0's auc: 0.924306
[5]	valid_0's l2: 0.168595	valid_0's auc: 0.929581
[6]	valid_0's l2: 0.156632	valid_0's auc: 0.930276
[7]	valid_0's l2: 0.145795	valid_0's auc: 0.932842
[8]	valid_0's l2: 0.136315	valid_0's auc: 0.932686
[9]	valid_0's l2: 0.127344	valid_0's auc: 0.933302
[10]	valid_0's l2: 0.119463	valid_0's auc: 0.933826
[11]	valid_0's l2: 0.112441	valid_0's auc: 0.939449
[12]	valid_0's l2: 0.105825	valid_0's auc: 0.939892
[13]	valid_0's l2: 0.0998609	valid_0's auc: 0.940266
[14]	valid_0's l2: 0.0945551	valid_0's auc: 0.940055
[15]	valid_0's l2: 0.0895879	valid_0's auc: 0.940081
[16]	valid_0's l2: 0.0850423	valid_0's auc: 0.941114
[17]	valid_0's l2: 0.0809772	valid_0's auc: 0.942017
[18]	valid_0's l2: 0.077288	valid_0's auc: 0.941993
[19]	valid_0's l2: 0.0741044	va

In [8]:
fpr, tpr, thresholds = roc_curve(dev_df['label'], np.mean(predict_results, axis=0))
print(auc(fpr, tpr))

0.9577708176521962


In [67]:
import tensorflow as tf
from sklearn import preprocessing

In [107]:
training_temp_df = pd.concat([training_five_folds[0], training_five_folds[1], training_five_folds[2], training_five_folds[3], training_five_folds[4]])

In [108]:
scaler = preprocessing.StandardScaler().fit(training_temp_df.drop(drop_features, axis=1))

In [74]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(features.shape[0]).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [13]:
my_feature_columns = []
for key in dev_df.drop(drop_features, axis=1).keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [166]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 5],
    n_classes=2,
    dropout=0.2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x14eff5eb8>, '_model_dir': '/var/folders/8n/2s3zkz2j7dq1d6c0vyldl7l80000gn/T/tmptvx3wrka', '_save_checkpoints_steps': None, '_tf_random_seed': None, '_task_id': 0, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_evaluation_master': '', '_master': '', '_session_config': None, '_service': None}


In [167]:
used_columns = training_temp_df.drop(drop_features, axis=1).columns.tolist()
training_temp_df2 = training_temp_df.copy()
training_temp_df2[used_columns] = scaler.fit_transform(training_temp_df.drop(drop_features, axis=1))

classifier.train(
    input_fn=lambda:train_input_fn(training_temp_df2.drop(drop_features, axis=1), training_temp_df['label'], 
                                   100),
    steps=3000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/8n/2s3zkz2j7dq1d6c0vyldl7l80000gn/T/tmptvx3wrka/model.ckpt.
INFO:tensorflow:loss = 67.78418, step = 1
INFO:tensorflow:global_step/sec: 91.5427
INFO:tensorflow:loss = 16.402182, step = 101 (1.093 sec)
INFO:tensorflow:global_step/sec: 172.718
INFO:tensorflow:loss = 11.738552, step = 201 (0.579 sec)
INFO:tensorflow:global_step/sec: 174.21
INFO:tensorflow:loss = 13.350077, step = 301 (0.574 sec)
INFO:tensorflow:global_step/sec: 52.5857
INFO:tensorflow:loss = 17.786304, step = 401 (1.902 sec)
INFO:tensorflow:global_step/sec: 168.568
INFO:tensorflow:loss = 13.732536, step = 501 (0.593 sec)
INFO:tensorflow:global_step/sec: 175.475
INFO:tensorflow:loss = 22.07432, step = 601 (0.570 sec)
INFO:tensorflow:

In [168]:
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(training_temp_df2.drop(drop_features, axis=1), training_temp_df['label'], 
                                  100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-18-14:11:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/8n/2s3zkz2j7dq1d6c0vyldl7l80000gn/T/tmptvx3wrka/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-18-14:11:40
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.9501324, accuracy_baseline = 0.8914874, auc = 0.96452355, auc_precision_recall = 0.8228087, average_loss = 0.1351346, global_step = 3000, label/mean = 0.1085126, loss = 13.47697, prediction/mean = 0.10781629


In [115]:
print('accuracy: {accuracy:0.3f}'.format(**eval_result))

accuracy: 0.952


In [81]:
predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(training_temp_df,
                                labels=None,
                                batch_size=100))

In [82]:
predictions

<generator object predict at 0x12026ef78>

In [83]:
predict_probability = []
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][1]
    predict_probability.append(probability)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/8n/2s3zkz2j7dq1d6c0vyldl7l80000gn/T/tmp7w5jmk7c/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [84]:
predict_probability

[0.5022954,
 0.5022954,
 0.5022954,
 0.5022954,
 0.42395657,
 0.5022954,
 0.23010153,
 0.003186701,
 0.0016014689,
 0.01548331,
 0.0012763458,
 0.40929434,
 1.1369972e-05,
 0.062302403,
 0.007328378,
 0.5022954,
 0.0027356888,
 0.01182208,
 0.0035884548,
 0.014065783,
 0.025350096,
 0.5022954,
 0.5022954,
 0.026947618,
 0.05108192,
 0.0008468723,
 0.5022954,
 0.00020328387,
 0.02022051,
 0.5022954,
 0.0010660788,
 0.024720177,
 0.02309833,
 0.02848085,
 1.0666582e-05,
 0.00048582457,
 0.0088824695,
 7.1045242e-06,
 0.003147936,
 0.0012802028,
 0.05201002,
 0.0162293,
 0.014583045,
 0.0033879536,
 0.5022954,
 0.0002988395,
 0.5022954,
 0.0008185586,
 0.034217905,
 0.013473118,
 0.0027507502,
 0.037740827,
 0.0024516047,
 0.0031014746,
 0.008443124,
 1.08640825e-05,
 0.002035162,
 0.016220272,
 0.5022954,
 0.1217303,
 0.013232487,
 0.00019743523,
 0.0001408894,
 0.016899917,
 0.0153549425,
 0.0154157495,
 0.5022954,
 0.03482506,
 0.0074339374,
 0.011475935,
 0.010695267,
 0.0010650083,
 

In [85]:
fpr, tpr, thresholds = roc_curve(training_five_folds[0]['label'], predict_probability)
print(auc(fpr, tpr))

0.9470121808524149


In [169]:
used_columns = training_five_folds[0].drop(drop_features, axis=1).columns.tolist()
dev_temp_df = dev_df.copy()
dev_temp_df[used_columns] = scaler.fit_transform(dev_temp_df.drop(drop_features, axis=1))

predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(dev_temp_df.drop(drop_features, axis=1),
                                labels=None,
                                batch_size=100))
predict_probability = []
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][1]
    predict_probability.append(probability)
    
fpr, tpr, thresholds = roc_curve(dev_df['label'], predict_probability)
print(auc(fpr, tpr))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/8n/2s3zkz2j7dq1d6c0vyldl7l80000gn/T/tmptvx3wrka/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0.9534895062310662


In [163]:
print(np.sum(dev_df['label']==1))
print(dev_df.shape)
error_df = dev_df[((np.array(predict_probability)>=0.5)-dev_df['label'])!=0]
print(error_df.shape)
error_df.head(15)

1209
(12810, 93)
(712, 93)


,FileID,label,fold,affect_user_count,total_event_count,each_user_event_avg,each_user_event_std,each_user_event_rstd,first_time_occur,last_time_occur,time_duration,first_24h_count,first_24h_ratio,first_24h_affect_user_count,first_24h_affect_user_ratio,first_24h_each_user_event_avg,bin_0_count,bin_0_ratio,bin_0_affect_user_count,bin_0_affect_user_ratio,bin_0_each_user_event_avg,bin_1_count,bin_1_ratio,bin_1_affect_user_count,bin_1_affect_user_ratio,bin_1_each_user_event_avg,bin_2_count,bin_2_ratio,bin_2_affect_user_count,bin_2_affect_user_ratio,bin_2_each_user_event_avg,bin_3_count,bin_3_ratio,bin_3_affect_user_count,bin_3_affect_user_ratio,bin_3_each_user_event_avg,bin_4_count,bin_4_ratio,bin_4_affect_user_count,bin_4_affect_user_ratio,bin_4_each_user_event_avg,bin_5_count,bin_5_ratio,bin_5_affect_user_count,bin_5_affect_user_ratio,bin_5_each_user_event_avg,bin_6_count,bin_6_ratio,bin_6_affect_user_count,bin_6_affect_user_ratio,bin_6_each_user_event_avg,bin_7_count,bin_7_ratio,bin_7_affect_user_count,bin_7_affect_user_ratio,bin_7_each_user_event_avg,bin_8_count,bin_8_ratio,bin_8_affect_user_count,bin_8_affect_user_ratio,bin_8_each_user_event_avg,bin_9_count,bin_9_ratio,bin_9_affect_user_count,bin_9_affect_user_ratio,bin_9_each_user_event_avg,customer_spread_time_mean,customer_spread_time_std,customer_spread_time_rstd,event_diff_time_mean,event_diff_time_median,event_diff_time_std,event_diff_time_rstd,customer_virus_ratio_avg,customer_virus_ratio_min,customer_virus_ratio_max,customer_virus_ratio_median,customer_virus_ratio_std,customer_virus_ratio_rstd,product_virus_ratio_avg,product_virus_ratio_min,product_virus_ratio_max,product_virus_ratio_median,product_virus_ratio_std,product_virus_ratio_rstd,customer_has_static_ratio,customer_has_static_b3_ratio,customer_has_static_count,customer_has_static_b3_count,customer_virus_ratio_wavg,customer_virus_zero_count,customer_virus_zero_ratio,customer_virus_zero_ratio2
14,006c27781f4d8e826102151c25e16b0f,1,0,5,118,23.600000,32.569925,1.380082,1492217701,1492601821,384120,6,0.050847,2,0.400000,3.000000,6,0.050847,2,0.400000,3.000000,0,0.000000,0,0.000000,0.00,0,0.000000,0,0.000000,0.0,0,0.000000,0,0.000000,0.000,0,0.000000,0,0.000000,0.0,31,0.262712,3,0.600000,10.333333,12,0.101695,2,0.400000,6.000000,22,0.186441,2,0.400000,11.0000,13,0.110169,2,0.400000,6.500000,34,0.288136,2,0.400000,17.0,40110.400000,86529.312625,2.157279,3126.796610,3584.5,6535.290570,2.090091,0.031394,0.000000,0.156970,0.000000,0.062788,2.000000,0.117912,0.087423,0.156970,0.109343,0.029032,0.246216,0.400000,0.200000,2.0,1.0,0.156960,1.0,0.200000,0.500000
27,00bf11cf3da869516c332fb98a52713e,1,0,96,320,3.333333,17.138241,5.141472,1491817460,1492293907,476447,128,0.400000,27,0.281250,4.740741,49,0.153125,9,0.093750,5.444444,89,0.278125,25,0.260417,3.56,15,0.046875,2,0.020833,7.5,34,0.106250,16,0.166667,2.125,25,0.078125,10,0.104167,2.5,39,0.121875,20,0.208333,1.950000,30,0.093750,15,0.156250,2.000000,27,0.084375,16,0.166667,1.6875,5,0.015625,3,0.031250,1.666667,7,0.021875,7,0.072917,1.0,4628.510417,9701.063668,2.095936,11164.540625,187.5,45241.275462,4.052229,0.003656,0.000000,0.250000,0.000000,0.027275,7.459843,0.082771,0.000000,0.139611,0.094213,0.044150,0.533400,0.270833,0.093750,26.0,9.0,0.099605,24.0,0.250000,0.923077
28,00c53dcce81a8ba2428ee204ecd68185,1,0,3,26,8.666667,8.144528,0.939753,1491994048,1492346510,352462,23,0.884615,3,1.000000,7.666667,5,0.192308,1,0.333333,5.000000,0,0.000000,0,0.000000,0.00,18,0.692308,2,0.666667,9.0,0,0.000000,0,0.000000,0.000,0,0.000000,0,0.000000,0.0,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.0000,0,0.000000,0,0.000000,0.000000,3,0.115385,1,0.333333,3.0,28343.666667,46411.801197,1.637466,10408.346154,5.0,52406.704918,5.035066,0.095225,0.000000,0.156970,0.128705,0.068316,0.717415,0.131673,0.109343,0.156970,0.128705,0.019556,0.148523,0.666667,0.666667,2.0,2.0,0.154886,0.0,0.000000,0.000000
34,00e5afed4af58cd2d8818583c5697116,1,0,5,637,127.400

In [157]:
dev_df[dev_df['label']==0]['event_diff_time_median'].quantile(0.8)

70.0

In [158]:
dev_df[dev_df['label']==1]['event_diff_time_median'].quantile(0.8)

88.0

In [217]:
import numpy as np
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout

drop_features2 = ['fold', 'label', 'FileID', 'first_time_occur', 'last_time_occur', 'customer_has_static_ratio', \
                'customer_has_static_b3_ratio', 'customer_has_static_count', 'customer_has_static_b3_count', \
                'customer_virus_ratio_avg', 'customer_virus_ratio_min', 'customer_virus_ratio_max', \
                'customer_virus_ratio_median', 'customer_virus_ratio_std', 'customer_virus_ratio_rstd', \
                'product_virus_ratio_avg', 'product_virus_ratio_min', 'product_virus_ratio_max', \
                'product_virus_ratio_median', 'product_virus_ratio_std', 'product_virus_ratio_rstd', \
                 'customer_virus_ratio_wavg', 'customer_virus_zero_count', 'customer_virus_zero_ratio', 'customer_virus_zero_ratio2']

model = Sequential()
model.add(Dense(3, input_dim=training_temp_df2.drop(drop_features2, axis=1).shape[1], activation='relu', 
                kernel_regularizer=regularizers.l2(0.00)))
#model.add(Dropout(0.2))
model.add(Dense(3, activation='relu', kernel_regularizer=regularizers.l2(0.00)))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(training_temp_df2.drop(drop_features2, axis=1), training_temp_df['label'],
          epochs=50,
          batch_size=128)
score = model.evaluate(training_temp_df2.drop(drop_features2, axis=1), training_temp_df['label'], batch_size=128)

Epoch 1/50
33609/33609 [==============================] - 1s 34us/step - loss: 0.3690 - acc: 0.8711
Epoch 2/50
33609/33609 [==============================] - 0s 13us/step - loss: 0.2926 - acc: 0.8892
Epoch 3/50
33609/33609 [==============================] - 0s 14us/step - loss: 0.2842 - acc: 0.8897
Epoch 4/50
33609/33609 [==============================] - 0s 13us/step - loss: 0.2801 - acc: 0.8899
Epoch 5/50
33609/33609 [==============================] - 0s 13us/step - loss: 0.2773 - acc: 0.8901
Epoch 6/50
33609/33609 [==============================] - 0s 14us/step - loss: 0.2751 - acc: 0.8902
Epoch 7/50
33609/33609 [==============================] - 0s 13us/step - loss: 0.2734 - acc: 0.8901
Epoch 8/50
33609/33609 [==============================] - 0s 13us/step - loss: 0.2718 - acc: 0.8902
Epoch 9/50
33609/33609 [==============================] - 0s 13us/step - loss: 0.2705 - acc: 0.8903
Epoch 10/50
33609/33609 [==============================] - 0s 13us/step - loss: 0.2694 - acc: 0.8904

In [218]:
predict_probability = model.predict(training_temp_df2.drop(drop_features2, axis=1), batch_size=100)
fpr, tpr, thresholds = roc_curve(training_temp_df['label'], predict_probability)
print(auc(fpr, tpr))

0.8526691070365393


In [219]:
predict_probability = model.predict(dev_temp_df.drop(drop_features2, axis=1), batch_size=100)
fpr, tpr, thresholds = roc_curve(dev_temp_df['label'], predict_probability)
print(auc(fpr, tpr))

0.8804136775807738


In [220]:
used_columns = training_five_folds[0].drop(drop_features2, axis=1).columns.tolist()
test_temp_df = test_df.copy()
test_temp_df[used_columns] = scaler.fit_transform(test_temp_df.drop(drop_features2, axis=1))

predict_probability = model.predict(test_temp_df.drop(drop_features2, axis=1), batch_size=100)
fpr, tpr, thresholds = roc_curve(test_temp_df['label'], predict_probability)
print(auc(fpr, tpr))

0.8620118743640447
